## Частина 2

​

В цій частині ми знову будемо працювати з датасетом fashion_mnist.



На відміну від попереднього завдання вам пропонується створити згорткову нейромережу, що використовує VGG16 в якості згорткової основи.



Навчіть отриману мережу на даних із датасету fashion_mnist. Спробуйте досягти максимально можливої точності класифікації за рахунок маніпуляції параметрами мережі. Під час навчання використовуйте прийоми донавчання та виділення ознак.



Порівняйте точність отриманої згорткової мережі з точністю багатошарової мережі з попереднього завдання. Зробіть висновки.

In [24]:
import numpy as np
import tensorflow as tf
# from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers

fashion_mnist = tf.keras.datasets.fashion_mnist


In [15]:
# Завантаження датасету Fashion MNIST
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [10]:

# Перевірка форми даних
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

Train images shape: (60000, 28, 28)
Train labels shape: (60000,)
Test images shape: (10000, 28, 28)
Test labels shape: (10000,)


In [16]:
# Зміна розмірності зображень на 32x32
train_images = tf.image.resize(train_images[..., tf.newaxis], [32, 32])
test_images = tf.image.resize(test_images[..., tf.newaxis], [32, 32])

In [17]:
# # Збільшення кількості каналів до 3 для сумісності з VGG16
# x_train = np.stack([x_train]*3, axis=-1)
# x_test = np.stack([x_test]*3, axis=-1)

# # Зміна розміру зображень до 224x224 для сумісності з VGG16
# x_train = tf.image.resize(x_train, (224, 224))
# x_test = tf.image.resize(x_test, (224, 224))

# Збільшення кількості каналів у зображеннях з 1 до 3
train_images = tf.repeat(train_images, 3, axis=-1)
test_images = tf.repeat(test_images, 3, axis=-1)

In [18]:
print(train_images.shape)
print(test_images.shape)

(60000, 32, 32, 3)
(10000, 32, 32, 3)


In [19]:
# # Нормалізація даних
# x_train = preprocess_input(x_train)
# x_test = preprocess_input(x_test)

# # One-hot енкодінг міток
# y_train = to_categorical(y_train, 10)
# y_test = to_categorical(y_test, 10)

# Нормалізація піксельних значень до діапазону [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

In [20]:
# Створення об'єкта генератора даних з параметрами аугментації для навчальних даних
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Генератор даних для навчального набору
train_generator = train_datagen.flow(train_images, train_labels, batch_size=64)

# Генератор даних для валідаційного набору без аугментації
validation_datagen = ImageDataGenerator()

# Генератор даних для валідаційного набору
validation_generator = validation_datagen.flow(test_images, test_labels, batch_size=64)

In [21]:
# Завантаження VGG16
conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
conv_base.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [22]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [25]:
# Визначення моделі
model = models.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dense(10, activation="softmax"), # Зміна на 10 вихідних класів, якщо це класифікація для Fashion MNIST
])


In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 14848586 (56.64 MB)
Trainable params: 133898 (523.04 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [28]:

# Компіляція моделі
model.compile(
    loss="sparse_categorical_crossentropy", # Зміна на sparse_categorical_crossentropy для багатокласової класифікації
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
    metrics=["accuracy"]
)


In [29]:

# Тренування моделі
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=100, # Зменшення кількості епох для швидшого тестування
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/100
100/100 [==============================] - 107s 1s/step - loss: 2.3008 - accuracy: 0.1273 - val_loss: 2.1927 - val_accuracy: 0.1744
Epoch 2/100
100/100 [==============================] - 111s 1s/step - loss: 2.0939 - accuracy: 0.2856 - val_loss: 2.0042 - val_accuracy: 0.3609
Epoch 3/100
100/100 [==============================] - 153s 2s/step - loss: 1.9393 - accuracy: 0.4616 - val_loss: 1.8492 - val_accuracy: 0.4909
Epoch 4/100
100/100 [==============================] - 130s 1s/step - loss: 1.8010 - accuracy: 0.5320 - val_loss: 1.7109 - val_accuracy: 0.5847
Epoch 5/100
100/100 [==============================] - ETA: 0s - loss: 1.6713 - accuracy: 0.5828

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(20, 7), dpi=80)
plt.grid(True)

plt.plot(epochs, acc, "bo", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")

plt.title("Training and validation accuracy")
plt.legend()

plt.figure(figsize=(20, 7), dpi=80)
plt.grid(True)

plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")

plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Отримання передбачень для тестових даних
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Генерація classification report
report = classification_report(test_labels, predicted_classes)
print(report)

# Перевірка точності на тестових даних за допомогою генератора
test_loss, test_acc = model.evaluate(validation_generator, steps=len(test_images) // 64, verbose=2)
print(f'Test accuracy: {test_acc}')

In [ ]:
model.save("model_VGG16")

## 2. Донавчання моделі

In [ ]:
conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))

In [ ]:

# розмороження кілька верхніх шарів у створеному раніше conv_base
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == "block5_conv1":
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# Визначаємо модифікову модель
modified_model = models.Sequential([
   conv_base,
   layers.Flatten(),
   layers.Dense(256, activation="relu"),
   layers.Dense(1, activation="sigmoid"),
])


In [ ]:

# Компілюємо модель
model.compile(
    loss="sparse_categorical_crossentropy", # Зміна на sparse_categorical_crossentropy для багатокласової класифікації
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
    metrics=["accuracy"]
)

In [ ]:
# Навчаємо модель
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=50
)

In [ ]:
# Отримання передбачень для тестових даних
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Генерація classification report
report = classification_report(test_labels, predicted_classes)
print(report)

# Перевірка точності на тестових даних за допомогою генератора
test_loss, test_acc = model.evaluate(validation_generator, steps=len(test_images) // 64, verbose=2)
print(f'Test accuracy: {test_acc}')

In [ ]:
import matplotlib.pyplot as plt

acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(20, 7), dpi=80)
plt.grid(True)

plt.plot(epochs, acc, "bo", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")

plt.title("Training and validation accuracy")
plt.legend()

plt.figure(figsize=(20, 7), dpi=80)
plt.grid(True)

plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")

plt.legend()
plt.show()

In [ ]:
modified_model.save("modified_model_VGG16")

**Висновок:** Застосування VGG16 в якості згорткової основи для датасету алгоритмів генератора та подальне донавчання створеної мережі показало покращення точності і інших метрик якості, але неістотно. ТОбто по суті донавчання відбулося на базі вже досягнугнутої точності, але покразилось незначно, навіть беруи під увагу велику кількість епох. Варто відмінити те, що модель, навіть при такій великій кількості епох не стала перенавченою. Але залишилась з відносно невисокими показниками точності - бльзько 0.78.

In [ ]:
# Завантаження моделі VGG16 без верхніх шарів
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Замороження ваг шарів базової моделі
for layer in vgg_base.layers:
    layer.trainable = False

# Додавання власного класифікатора
model = tf.keras.Sequential([
    vgg_base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Аугментація даних
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Підготовка генератора даних
train_generator = datagen.flow(x_train, y_train, batch_size=32)

# Навчання моделі
history = model.fit(train_generator, epochs=20, validation_data=(x_test, y_test))


In [ ]:
def plot_history(history_object):
  # Побудова графіку точності
  plt.figure(figsize=(12, 4))
  plt.subplot(1, 2, 1)
  plt.plot(history_object.history['accuracy'], label='Точність на тренуванні')
  plt.plot(history_object.history['val_accuracy'], label='Точність на валідації')
  plt.xlabel('Епоха')
  plt.ylabel('Точність')
  plt.title('Точність тренування та валідації')
  plt.legend()

  # Додавання тестової точності до графіку
  plt.scatter(len(history_object.history['accuracy']), test_acc, label='Точність на тесті', color='red')
  plt.legend()

  # Побудова графіку втрат
  plt.subplot(1, 2, 2)
  plt.plot(history_object.history['loss'], label='Втрати на тренуванні')
  plt.plot(history_object.history['val_loss'], label='Втрати на валідації')
  plt.xlabel('Епоха')
  plt.ylabel('Втрати')
  plt.title('Втрати тренування та валідації')
  plt.legend()

# приклад використання
# plot_history(history_1)

In [ ]:
plot_history(history)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Припустимо, що history - це об'єкт History, отриманий після тренування моделі
# history = model.fit(...)
def plot_history_df(history):
  # Перетворюємо дані з об'єкта History в DataFrame
  h_df = pd.DataFrame(history.history)

  # Додаємо стовпець для епох
  h_df['epoch'] = h.epoch

  # Відображаємо таблицю
  print(h_df)

  # Візуалізація метрик
  plt.figure(figsize=(12, 6))

  # Втрати
  plt.subplot(1, 2, 1)
  plt.plot(h_df['epoch'], h_df['loss'], label='Train Loss')
  plt.plot(h_df['epoch'], h_df['val_loss'], label='Validation Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('Train and Validation Loss')
  plt.legend()

  # Точність
  plt.subplot(1, 2, 2)
  plt.plot(h_df['epoch'], h_df['accuracy'], label='Train Accuracy')
  plt.plot(h_df['epoch'], h_df['val_accuracy'], label='Validation Accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.title('Train and Validation Accuracy')
  plt.legend()

  plt.tight_layout()
  plt.show()

In [ ]:
# Оцінка на тестових даних
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')


In [ ]:
# Отримання передбачень для тестових даних
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Отримання індексу класу для кожного зображення
true_classes = np.argmax(test_labels, axis=1)

# Генерація classification report
report = classification_report(true_classes, predicted_classes)

# Вивід результатів
print(report)

In [ ]:
model.save ("fashion_mnist_model.keras")

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd

def parse_classification_report(report):
    report_dict = {}
    lines = report.split('\n')

    # Зчитування класів та метрик
    for line in lines[2:-4]:
        parts = line.split()
        class_name = parts[0]
        precision, recall, f1_score, support = map(float, parts[1:])
        report_dict[class_name] = {
            'precision': precision,
            'recall': recall,
            'f1-score': f1_score,
            'support': support
        }

    # Зчитування загальних метрик
    accuracy = float(lines[-4].split()[1])
    report_dict['accuracy'] = accuracy

    macro_avg_parts = lines[-2].split()[1:]
    report_dict['macro avg'] = {
        'precision': float(macro_avg_parts[0]),
        'recall': float(macro_avg_parts[1]),
        'f1-score': float(macro_avg_parts[2]),
        'support': float(macro_avg_parts[3])
    }

    weighted_avg_parts = lines[-1].split()[1:]
    report_dict['weighted avg'] = {
        'precision': float(weighted_avg_parts[0]),
        'recall': float(weighted_avg_parts[1]),
        'f1-score': float(weighted_avg_parts[2]),
        'support': float(weighted_avg_parts[3])
    }

    return report_dict

# Приклад використання
# Припустимо, у нас є справжні значення та передбачення моделі
y_true = [0, 1, 0, 1, 0, 1, 0, 1]
y_pred_model_1 = [0, 1, 0, 0, 0, 1, 1, 1]
y_pred_model_2 = [0, 1, 0, 1, 0, 1, 0, 1]

# Генерація classification_report для кожної моделі
report_model_1 = classification_report(y_true, y_pred_model_1)
report_model_2 = classification_report(y_true, y_pred_model_2)

# Розпарсення звітів у словники
parsed_report_model_1 = parse_classification_report(report_model_1)
parsed_report_model_2 = parse_classification_report(report_model_2)

# Порівняння звітів
def report_to_df(report):
    report_data = []
    for label, metrics in report.items():
        if isinstance(metrics, dict):
            report_data.append([label] + list(metrics.values()))
        else:
            report_data.append([label, metrics, None, None, None])
    columns = ['class', 'precision', 'recall', 'f1-score', 'support']
    return pd.DataFrame(report_data, columns=columns)

df_model_1 = report_to_df(parsed_report_model_1)
df_model_2 = report_to_df(parsed_report_model_2)

comparison_df = df_model_1.set_index('class').join(df_model_2.set_index('class'), lsuffix='_model_1', rsuffix='_model_2')

# Вивід порівняння
print(comparison_df)
